In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

def preprocess_data_from_file(file_path):
    # Initialize lists to store data
    features_list = []
    relevance_list = []

    # Read data from the file
    with open(file_path, 'r') as file:
        data = file.readlines()

    # Process each line of data
    for line in data:
        # Split by space
        parts = line.split(" ")
        # Extract qid
        qid = parts[0]  # Assuming qid is the first part of each line
        # Extract relevance (removing trailing newline character)
        relevance = parts[-1].strip()
        # Extract features
        features = parts[1:-4]  # Assuming features are between qid and relevance
        # Append to respective lists
        features_list.append(features)
        relevance_list.append(relevance)

    # Convert features to DataFrame
    features_df = pd.DataFrame(features_list)

    # Add qid and relevance columns
    features_df.insert(0, "qid", qid)
    features_df["relevance"] = relevance_list

    # Rename columns
    columns = {i: f"feature{i+1}" for i in range(len(features_df.columns)-1)}
    features_df.rename(columns=columns, inplace=True)

    return features_df



# Example usage:
file_path1 = '/content/drive/My Drive/IE_506_RANKING/S1.txt'  # Update with your file path
features_df1 = preprocess_data_from_file(file_path1)

In [4]:
def preprocess_data(data):
    rows = []
    for row in data:
        if '#' in row:
            row = row.split('#')[0].strip()  # Exclude the part after '#'
        if not row:
            continue  # Skip empty lines
        parts = row.strip().split(' ')  # Split row by spaces
        relevance = int(parts[0])  # First number is relevance
        qid = int(parts[1].split(':')[1])  # Extract qid
        features = {}
        for part in parts[2:]:
            idx, val = part.split(':')
            features[int(idx)] = float(val)
        row_dict = {'qid': qid, 'relevance': relevance}
        row_dict.update(features)  # Add features to the row
        rows.append(row_dict)
    return pd.DataFrame(rows)

# Read data from a text file
file_path1 = '/content/drive/My Drive/IE_506_RANKING/S1.txt'  # Update with your file path
with open(file_path1, 'r') as file:
    data1 = file.readlines()

# Example usage:
df1 = preprocess_data(data1)

In [5]:
# Print the last column
last_column = features_df1.iloc[:, -1]
# Add the last column to the DataFrame at the last position
df1['prob'] = last_column
# Convert the last column from string to float
df1['prob'] = df1['prob'].astype(float)

In [6]:
df1.head(125)

,qid,relevance,1,2,3,4,5,6,7,8,...,38,39,40,41,42,43,44,45,46,prob
0,10,0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.0,0.024691
1,10,1,0.031310,0.666667,0.50,0.166667,0.033206,0.0,0.0,0.0,...,0.686107,0.823908,0.750092,0.385426,0.923077,0.086207,0.333333,0.448276,0.0,0.416367
2,10,1,0.078682,0.166667,0.50,0.333333,0.080022,0.0,0.0,0.0,...,0.578581,0.868557,0.641385,0.010462,0.076923,0.074713,0.833333,0.678161,0.0,0.568950
3,10,1,0.019058,1.000000,1.00,0.500000,0.022591,0.0,0.0,0.0,...,0.868457,1.000000,0.863460,0.016642,0.153846,0.040230,0.833333,0.896552,0.0,0.775913
4,10,0,0.039477,0.000000,0.75,0.166667,0.040555,0.0,0.0,0.0,...,0.569440,0.769845,0.646567,0.073711,0.076923,0.034483,0.333333,0.218391,0.0,0.334800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,34,0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.078947,0.000000,0.000000,0.0,0.060317
121,34,0,0.074129,0.000000,0.00,0.000000,0.074129,0.0,0.0,0.0,...,0.695368,0.718155,0.647774,0.011770,0.150000,0.842105,0.571429,0.915254,0.0,0.136433
122,34,0,0.034099,0.000000,0.00,0.000000,0.034099,0.0,0.0,0.0,...,0.708105,0.710384,0.711672,1.000000,1.000000,0.342105,0.714286,0.576271,0.0,0.175514
123,34,0,0.074129,0.000000,0.00,0.000000,0.074129,0.0,0.0,0.0,...,0.707261,0.727017,0.660234,0.001154,0.050000,0.026316,0.571429,1.000000,0.0,0.158908


In [7]:
import pandas as pd
from itertools import permutations

def pairwise_ranking(df):
    output_data = []

    # Group the DataFrame by 'qid'
    grouped = df.groupby('qid')

    # Iterate over each group
    for qid, group in grouped:
        # Generate all permutations of pairs within the group
        pairs = permutations(group.iterrows(), 2)

        # Iterate over each pair
        for (idx1, doc1), (idx2, doc2) in pairs:
            # Extract relevance and prob values for each document
            relevance1, relevance2 = doc1['relevance'], doc2['relevance']
            prob1, prob2 = doc1['prob'], doc2['prob']

            # Extract features for doc1 and doc2
            features1 = doc1.drop(['qid', 'relevance', 'prob'])
            features2 = doc2.drop(['qid', 'relevance', 'prob'])

            # Determine output label based on relevance and prob values
            if relevance1 > relevance2:
                output_label = 1
            elif relevance1 < relevance2:
                output_label = 0
            else:
                output_label = 1 if prob1 > prob2 else 0

            # Append the result to the output data
            output_data.append({'qid': qid, 'doc1': idx1, 'doc2': idx2, 'output_label': output_label,
                                **{f'feature_{i}': features1[i] for i in range(1, 47)},
                                **{f'feature_{i+46}': features2[i] for i in range(1, 47)}})

    # Create DataFrame from output data
    output_df = pd.DataFrame(output_data)
    return output_df


In [8]:
# Example usage:
# Assuming 'df' is the DataFrame containing qid, features, and relevance level
output_df1 = pairwise_ranking(df1)

In [9]:
output_df1

,qid,doc1,doc2,output_label,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92
0,10,0,1,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.646890,0.686107,0.823908,0.750092,0.385426,0.923077,0.086207,0.333333,0.448276,0.0
1,10,0,2,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.649824,0.578581,0.868557,0.641385,0.010462,0.076923,0.074713,0.833333,0.678161,0.0
2,10,0,3,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.918403,0.868457,1.000000,0.863460,0.016642,0.153846,0.040230,0.833333,0.896552,0.0
3,10,0,4,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.565875,0.569440,0.769845,0.646567,0.073711,0.076923,0.034483,0.333333,0.218391,0.0
4,10,0,5,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.912987,0.851143,0.676743,0.871849,0.001705,0.076923,0.000000,0.500000,0.425287,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581381,1828,14012,14007,1,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.705032,0.983064,0.874043,0.983256,0.000118,0.000713,0.152174,0.500000,0.129252,0.0
581382,1828,14012,14008,1,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.253665,0.519413,0.497421,0.505797,0.000000,0.000000,0.782609,0.500000,0.081633,0.0
581383,1828,14012,14009,1,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.498324,0.665563,0.793696,0.682627,0.013283,0.001427,0.021739,0.500000,0.190476,0.0
581384,1828,14012,14010,0,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.603386,0.906406,0.929225,0.873596,0.000061,0.000713,0.000000,0.500000,0.115646,0.0


In [10]:
import tensorflow as tf

# List of columns to be dropped from merged_df
cols_to_drop = ['output_label', 'doc1', 'doc2']  # Add the column names you want to drop here

# Drop the specified columns from merged_df to get features
X_train = output_df1.drop(cols_to_drop, axis=1).values
y_train = output_df1['output_label'].values

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output probability P(i,j)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using all the data
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Now you can use the model for prediction



Epoch 1/10
18169/18169 [==============================] - 26s 1ms/step - loss: 0.6659 - accuracy: 0.6533
Epoch 2/10
18169/18169 [==============================] - 23s 1ms/step - loss: 0.5911 - accuracy: 0.6849
Epoch 3/10
18169/18169 [==============================] - 23s 1ms/step - loss: 0.5772 - accuracy: 0.6963
Epoch 4/10
18169/18169 [==============================] - 24s 1ms/step - loss: 0.5715 - accuracy: 0.7014
Epoch 5/10
18169/18169 [==============================] - 24s 1ms/step - loss: 0.5678 - accuracy: 0.7048
Epoch 6/10
18169/18169 [==============================] - 23s 1ms/step - loss: 0.5654 - accuracy: 0.7063
Epoch 7/10
18169/18169 [==============================] - 23s 1ms/step - loss: 0.5631 - accuracy: 0.7079
Epoch 8/10
18169/18169 [==============================] - 23s 1ms/step - loss: 0.5617 - accuracy: 0.7091
Epoch 9/10
18169/18169 [==============================] - 23s 1ms/step - loss: 0.5604 - accuracy: 0.7097
Epoch 10/10
18169/18169 [==============================

In [11]:
# Assuming X_test contains the features of new data for which you want to predict probabilities
probabilities = model.predict(X_train)

# Display probabilities
print(probabilities)


18169/18169 [==============================] - 17s 912us/step
[[4.3938533e-04]
 [1.2331881e-04]
 [1.1043103e-05]
 ...
 [5.3882545e-01]
 [3.8653988e-01]
 [5.7389796e-01]]


In [12]:
print(probabilities.shape)

(581386, 1)


In [13]:
# Assuming probabilities is the numpy array containing the probabilities
output_df1['probabilities'] = probabilities

In [14]:
output_df1

,qid,doc1,doc2,output_label,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,probabilities
0,10,0,1,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.686107,0.823908,0.750092,0.385426,0.923077,0.086207,0.333333,0.448276,0.0,0.000439
1,10,0,2,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.578581,0.868557,0.641385,0.010462,0.076923,0.074713,0.833333,0.678161,0.0,0.000123
2,10,0,3,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.868457,1.000000,0.863460,0.016642,0.153846,0.040230,0.833333,0.896552,0.0,0.000011
3,10,0,4,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.569440,0.769845,0.646567,0.073711,0.076923,0.034483,0.333333,0.218391,0.0,0.000231
4,10,0,5,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.851143,0.676743,0.871849,0.001705,0.076923,0.000000,0.500000,0.425287,0.0,0.000059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581381,1828,14012,14007,1,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.983064,0.874043,0.983256,0.000118,0.000713,0.152174,0.500000,0.129252,0.0,0.457360
581382,1828,14012,14008,1,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.519413,0.497421,0.505797,0.000000,0.000000,0.782609,0.500000,0.081633,0.0,0.822729
581383,1828,14012,14009,1,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.665563,0.793696,0.682627,0.013283,0.001427,0.021739,0.500000,0.190476,0.0,0.538825
581384,1828,14012,14010,0,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.906406,0.929225,0.873596,0.000061,0.000713,0.000000,0.500000,0.115646,0.0,0.386540


In [15]:
import pandas as pd

# Assuming merged_df is your DataFrame containing the 'doc1' and 'probabilities' columns

# Calculate the sum of probabilities for each unique value in the 'doc1' column
sum_probabilities = output_df1.groupby('doc1')['probabilities'].sum()

# Map the sum of probabilities back to the original DataFrame
output_df1['sum_probabilities'] = output_df1['doc1'].map(sum_probabilities)


In [16]:
output_df1

,qid,doc1,doc2,output_label,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,probabilities,sum_probabilities
0,10,0,1,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.823908,0.750092,0.385426,0.923077,0.086207,0.333333,0.448276,0.0,0.000439,0.924373
1,10,0,2,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.868557,0.641385,0.010462,0.076923,0.074713,0.833333,0.678161,0.0,0.000123,0.924373
2,10,0,3,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,1.000000,0.863460,0.016642,0.153846,0.040230,0.833333,0.896552,0.0,0.000011,0.924373
3,10,0,4,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.769845,0.646567,0.073711,0.076923,0.034483,0.333333,0.218391,0.0,0.000231,0.924373
4,10,0,5,0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.676743,0.871849,0.001705,0.076923,0.000000,0.500000,0.425287,0.0,0.000059,0.924373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581381,1828,14012,14007,1,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.874043,0.983256,0.000118,0.000713,0.152174,0.500000,0.129252,0.0,0.457360,22.603413
581382,1828,14012,14008,1,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.497421,0.505797,0.000000,0.000000,0.782609,0.500000,0.081633,0.0,0.822729,22.603413
581383,1828,14012,14009,1,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.793696,0.682627,0.013283,0.001427,0.021739,0.500000,0.190476,0.0,0.538825,22.603413
581384,1828,14012,14010,0,0.009058,0.0,0.0,0.0,0.009056,0.0,...,0.929225,0.873596,0.000061,0.000713,0.000000,0.500000,0.115646,0.0,0.386540,22.603413


In [17]:
import pandas as pd

def create_new_dataframe(df):
    # Select unique combinations of qid, doc1, and sum_probabilities
    unique_df = df[['qid', 'doc1', 'sum_probabilities']].drop_duplicates('doc1').reset_index(drop=True)

    return unique_df

# Example usage:
new_df = create_new_dataframe(output_df1)



In [18]:
new_df

,qid,doc1,sum_probabilities
0,10,0,0.924373
1,10,1,32.398029
2,10,2,25.295986
3,10,3,37.473770
4,10,4,20.844736
...,...,...,...
14008,1828,14008,9.616220
14009,1828,14009,18.172695
14010,1828,14010,22.329851
14011,1828,14011,18.588820


In [19]:
import pandas as pd

def merge_relevance_and_prob_columns(new_df, df1):
    # Concatenate the relevance and prob columns from df1 to new_df
    merged_df = pd.concat([new_df, df1[['relevance', 'prob']]], axis=1)

    return merged_df

# Example usage:
merged_new_df = merge_relevance_and_prob_columns(new_df, df1)

In [20]:
merged_new_df

,qid,doc1,sum_probabilities,relevance,prob
0,10,0,0.924373,0,0.024691
1,10,1,32.398029,1,0.416367
2,10,2,25.295986,1,0.568950
3,10,3,37.473770,1,0.775913
4,10,4,20.844736,0,0.334800
...,...,...,...,...,...
14008,1828,14008,9.616220,0,0.058078
14009,1828,14009,18.172695,0,0.165237
14010,1828,14010,22.329851,1,0.427653
14011,1828,14011,18.588820,1,0.158485


In [21]:
import numpy as np

def compute_dcg(df, k=15):
    dcg_scores = []
    for qid in df['qid'].unique():
        # Filter dataframe for the current qid
        qid_df = df[df['qid'] == qid]

        # Sort documents based on relevance label and probability score
        sorted_df = qid_df.sort_values(by=['relevance', 'prob'], ascending=[False, False])

        # Select top k documents
        top_k_df = sorted_df.head(k)

        # Calculate DCG for the selected documents
        dcg = np.sum((2 ** top_k_df['relevance'] - 1) / np.log2(np.arange(2, k + 2)))

        dcg_scores.append(dcg)

    return dcg_scores

# Call the function to compute DCG score for the ideal case
ideal_dcg_scores = compute_dcg(merged_new_df)


In [22]:
ideal_dcg_scores

[5.8613479980138035,
 9.08436179088239,
 3.0,
 1.6309297535714575,
 9.850382753865262,
 5.8613479980138035,
 10.215953061456498,
 1.0,
 8.35459994530971,
 3.0,
 5.8613479980138035,
 5.8613479980138035,
 2.1309297535714578,
 7.215324023249391,
 10.215953061456498,
 0.0,
 4.254494511770457,
 4.948459118879392,
 0.0,
 9.873207505156719,
 4.892789260714372,
 7.210318626022307,
 7.092740438166795,
 5.8613479980138035,
 4.543559338088345,
 4.948459118879392,
 0.0,
 8.516354018913372,
 4.822502283739475,
 6.823465818787765,
 0.0,
 11.75826623577259,
 5.6113479980138035,
 5.8613479980138035,
 7.516354018913372,
 4.561606311644851,
 4.822502283739475,
 2.9484591188793923,
 12.450488370301299,
 2.5616063116448506,
 7.566525813130329,
 2.1309297535714578,
 3.6379996393207477,
 5.8613479980138035,
 5.8613479980138035,
 16.046828874347394,
 5.8613479980138035,
 3.9534645161064765,
 3.304666305987414,
 17.584043994041412,
 7.899859146463662,
 15.506352565492755,
 1.6309297535714575,
 10.478602596493

In [23]:
import numpy as np

def compute_model_dcg(df, k=15):
    dcg_scores = []
    for qid in df['qid'].unique():
        # Filter dataframe for the current qid
        qid_df = df[df['qid'] == qid]

        # Sort documents based on sum_probabilities
        sorted_df = qid_df.sort_values(by='sum_probabilities', ascending=False)

        # Select top k documents
        top_k_df = sorted_df.head(k)

        # Calculate DCG for the selected documents using relevance column
        dcg = np.sum((2 ** top_k_df['relevance'] - 1) / np.log2(np.arange(2, k + 2)))

        dcg_scores.append(dcg)

    return dcg_scores

# Call the function to compute DCG score for the model ranking
model_dcg_scores = compute_model_dcg(merged_new_df)

In [24]:
model_dcg_scores

[4.628442343987579,
 6.469118263978649,
 3.0,
 1.3868528072345416,
 9.669140674265167,
 4.804226098405069,
 7.362915234029126,
 0.43067655807339306,
 1.216632784784506,
 0.0,
 5.572283171695916,
 4.62255429759892,
 0.9157957528856715,
 2.961410822038439,
 6.490014602299533,
 0.0,
 1.5530747177246143,
 2.096365480635563,
 0.0,
 7.333409991214679,
 0.8671944789536635,
 5.026875698824168,
 3.2803123154856815,
 5.572283171695916,
 3.645678748277589,
 0.8519408931803423,
 0.0,
 0.31546487678572877,
 2.6116345599996387,
 2.6590480118017594,
 0.0,
 1.8202861692436803,
 5.332405052362674,
 5.8613479980138035,
 0.6309297535714575,
 3.0122158147880542,
 2.729510254733578,
 1.92124707417873,
 5.013591002382963,
 0.5,
 5.279642067948914,
 0.25,
 1.6309297535714575,
 4.967768709405153,
 3.6156589298154462,
 13.42189415313103,
 5.6113479980138035,
 1.4438210612467655,
 1.2201861405678749,
 16.121629619825367,
 4.105966608038563,
 12.904797003185887,
 0.3333333333333333,
 6.18894119025284,
 4.8613479

In [25]:
import pandas as pd

# Extract unique qids from df1
unique_qids = df1['qid'].unique()

# Create a dataframe to store qids, model_dcg_scores, and ideal_dcg_scores
result_df = pd.DataFrame({'qid': unique_qids})

# Add model_dcg_scores to the result dataframe
result_df['model_dcg_scores'] = model_dcg_scores

# Add ideal_dcg_scores to the result dataframe (assuming you already have them calculated)
result_df['ideal_dcg_scores'] = ideal_dcg_scores


In [26]:
result_df

,qid,model_dcg_scores,ideal_dcg_scores
0,10,4.628442,5.861348
1,15,6.469118,9.084362
2,33,3.000000,3.000000
3,34,1.386853,1.630930
4,37,9.669141,9.850383
...,...,...,...
334,1787,0.315465,1.000000
335,1798,0.356207,1.000000
336,1823,2.561606,3.953465
337,1825,0.000000,0.000000


In [27]:
# Remove rows where either model_dcg_scores or ideal_dcg_scores are zero
result_df = result_df[(result_df['model_dcg_scores'] != 0) & (result_df['ideal_dcg_scores'] != 0)]

In [28]:
result_df

,qid,model_dcg_scores,ideal_dcg_scores
0,10,4.628442,5.861348
1,15,6.469118,9.084362
2,33,3.000000,3.000000
3,34,1.386853,1.630930
4,37,9.669141,9.850383
...,...,...,...
333,1783,0.612276,2.130930
334,1787,0.315465,1.000000
335,1798,0.356207,1.000000
336,1823,2.561606,3.953465


In [29]:
# Calculate the ratio of model_dcg_scores to ideal_dcg_scores
result_df['ndcg_ratio'] = result_df['model_dcg_scores'] / result_df['ideal_dcg_scores']

# Calculate the average NDCG ratio
avg_ndcg = result_df['ndcg_ratio'].mean()

# Print the average NDCG ratio
print("Average NDCG Ratio:", avg_ndcg)


Average NDCG Ratio: 0.5718894503551538


<ipython-input-29-223a0cc09692>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['ndcg_ratio'] = result_df['model_dcg_scores'] / result_df['ideal_dcg_scores']
